# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [2]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import scipy.misc
from lifelines import KaplanMeierFitter

cf.go_offline()

In [5]:
data = pd.read_csv('attrition.csv')

In [7]:
data.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,0,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,0,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,0,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,0,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,0,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [9]:
data=data.fillna(data.mean())

In [13]:
data.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [12]:
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [10]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

In [14]:
rates = survival(data, 'Gender', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Tenure and Gender')

In [15]:
rates = survival(data, 'Age', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Tenure and Age')

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [ ]:
#The plot that shows the age of the employee has many more lines, since there are many more ages than genders at this 
#company. Presumably each age line has less information associated with them than the larger male or female buckets,
#making their changes more dramatic as well. 

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [16]:
rates = survival(data, 'Gender', 'YearsSinceLastPromotion', 'Attrition')

rates.iplot(kind='line', xTitle='Time Since Last Promotion (Years)', yTitle='Attrition',
            title='Attrition Rates by Promotion and Gender')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [17]:
rates = survival(data, 'MaritalStatus', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Tenure and Marital Status')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [18]:
rates = survival(data, 'MaritalStatus', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Attrition',
            title='Attrition Rates by Age and Marital Status')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [20]:
data["genstat"] = data["MaritalStatus"].map(str) + data["Gender"]

In [22]:
rates = survival(data, 'genstat', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Tenure and Combined Marital Status and Gender')

## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [23]:
rates = survival(data, 'JobSatisfaction', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Job Statisfaction')

## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [24]:
rates = survival(data, 'Department', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Job Statisfaction')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [26]:
salesdept = data[data['Department']=='Sales']

In [28]:
rates = survival(salesdept, 'JobRole', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Tenure at Company (Years)', yTitle='Attrition',
            title='Attrition Rates by Tenure and Job Role')

## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [30]:
data['HourlyGrade']=pd.qcut(data['HourlyRate'], 5, labels=["Very Low", "Low", "Moderate", "High","Very High"])

In [41]:
rates = survival(data, 'HourlyGrade', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Attrition',
            title='Attrition Rates by Age and Hourly Pay Grade')

## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.

In [33]:
import numpy as np

In [36]:
data['Demands']=np.where(data['OverTime']=='Yes', 1, 0)

In [39]:
rates = survival(data, 'Demands', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age', yTitle='Attrition',
            title='Attrition Rates by Age and Demands')